## Imports

In [6]:
from __future__ import print_function, division

from keras.layers import Input, Dense, Reshape, Flatten, Dropout, multiply, Lambda
from keras.layers import BatchNormalization, Activation, Embedding
from keras.layers.merge import concatenate
from keras.layers.advanced_activations import LeakyReLU
from keras.models import Sequential, Model
from keras.optimizers import Adam
from keras.constraints import NonNeg
from keras.utils.generic_utils import get_custom_objects
from keras import backend as K

import matplotlib.pyplot as plt

import pandas as pd
import numpy as np
import tensorflow as tf

In [7]:
BATCH_SIZE=64

## Custom losses

In [6]:
def customDiscriminatorLoss(tTrue,tPred):
    return K.sum(K.binary_crossentropy(tTrue[0],tPred[0]))

In [7]:
def customGeneratorLoss(yTrue,yPred,a=1):
    return a*K.sum((yTrue[:,-1]-yPred[:,-1])**2)

In [8]:
def customCombinedLoss(ytTrue,ytPred,a=1):
    return K.sum(customDiscriminatorLoss(ytTrue[1],ytPred[1]),customGeneratorLoss(ytTrue[0],ytPred[0],a))

In [9]:
def custom_activation(x):
    cond = K.greater(x,0)
    return K.switch(cond,x,K.minimum(-100.,x))

get_custom_objects().update({'custom_activation': Activation(custom_activation)})

## G_DG block

In [34]:
def get_values(x):
    outcomes, indices = x

    r = K.arange(K.shape(indices)[0])
    tiint = K.reshape(K.cast(indices,dtype='int32'),(-1,))
    idx = K.cast(K.stack([r,tiint],axis=-1),dtype='int32')
    
    return K.batch_flatten(Lambda(lambda x: tf.gather_nd(x[0],x[1]))([outcomes,idx]))


class G_DG():
    def __init__(self,data):
        
        self.data = data
        self.pat_features = self.data.n_features
        self.pat_shape = (self.pat_features,)
        self.num_treatments = 2
        self.latent_dim = self.num_treatments-1
        self.a = 0.1
        
        self.optimizer = Adam(0.0002, 0.5)
        
#         self.check_intermediate_layers()
        self.build_model()

    def check_intermediate_layers(self):
        # Build and compile the discriminator
        self.discriminator = self.build_discriminator()
        self.discriminator.compile(loss=['binary_crossentropy'],
                                   optimizer=self.optimizer)

        # Build the generator
        self.generator = self.build_generator()

        pat = Input(shape=(self.pat_features,))
        treatment = Input(shape=(self.num_treatments,))
        outcome = Input(shape=(1,))
        noise = Input(shape=(self.latent_dim,))
        treatment_indices = Input(shape=(1,),dtype='int32')
        
        gen_y = self.generator([pat, treatment, outcome, noise])                
        
        y_bar = Lambda(self.replace_values)([gen_y, treatment_indices, outcome])
        
        z=Model([pat,treatment,outcome,noise,treatment_indices],gen_y)
        
        pat = np.arange(self.pat_features * BATCH_SIZE).reshape((BATCH_SIZE,self.pat_features))
        treatment = one_hot(np.random.randint(0,2,BATCH_SIZE))
        outcome = np.random.randint(0,2,(BATCH_SIZE,1))
        noise = np.random.uniform(-1,2,(BATCH_SIZE,self.latent_dim))
        treatment_indices = np.argmax(treatment,axis=1)

        gen_y = z.predict([pat,treatment,outcome,noise,treatment_indices])
        ybar = zz.predict([pat,treatment,outcome,noise,treatment_indices])
        
    def build_model(self):
        
        # Build and compile the discriminator
        self.discriminator = self.build_discriminator()
#         self.discriminator.compile(loss=['binary_crossentropy'],
#                                    optimizer=self.optimizer)

        # Build the generator
        self.generator = self.build_generator()

        pat = Input(shape=(self.pat_features,))
        treatment = Input(shape=(self.num_treatments,))
        outcome = Input(shape=(1,))
        noise = Input(shape=(self.latent_dim,))
        treatment_indices = Input(shape=(1,),dtype='int32')
        
        gen_y = self.generator([pat, treatment, outcome, noise])                
        
        predicted_outcome_of_real_treatment = Lambda(get_values,name='predicted_outcome_of_real_treatment')([gen_y,treatment_indices])

        y_bar = Lambda(self.replace_values,name='y_bar')([gen_y, treatment_indices, outcome])
        
        valid = self.discriminator([pat, y_bar])        
        
        self.combined = Model([pat, treatment, outcome, noise, treatment_indices], [valid, predicted_outcome_of_real_treatment])
        
        self.combined.compile(loss=['binary_crossentropy','mean_squared_error'],
                              loss_weights=[1.,self.a],
                              optimizer=self.optimizer)
        
    def replace_values(self,x):
        outs, indices, values = x

        #this is due to a strange bug between lambda and integers....
        indices = K.cast(indices, 'int32')

        #create one_hot indices
        one_hot_indices = K.one_hot(indices, self.num_treatments) #size is the size of gen_out
        one_hot_indices = K.batch_flatten(one_hot_indices)

        #have the desired values at their correct positions
        values_to_use = one_hot_indices * values

        #if values are 0, use gen_out, else use values
        return K.switch(K.equal(values_to_use, 0), outs, values_to_use)
        
    def build_generator(self):

        model = Sequential()

        model.add(Dense(512, input_dim=self.pat_features+self.num_treatments+1+self.latent_dim))
        model.add(LeakyReLU(alpha=0.2))
        model.add(BatchNormalization(momentum=0.6))
        model.add(Dense(1024))
        model.add(LeakyReLU(alpha=0.2))
        model.add(BatchNormalization(momentum=0.6))
        model.add(Dense(256))
        model.add(LeakyReLU(alpha=0.2))
        model.add(BatchNormalization(momentum=0.6))  
        model.add(Dense(self.num_treatments, activation='hard_sigmoid'))
#         model.summary()

        pat = Input(shape=(self.pat_features,),dtype='float32')
        treatment = Input(shape=(self.num_treatments,),dtype='float32')
        real_outcome = Input(shape=(1,), dtype='float32')
        noise = Input(shape=(self.latent_dim,),dtype='float32')

        model_input = concatenate([pat, treatment, real_outcome, noise])

        outcomes = model(model_input)
        
        return Model([pat, treatment, real_outcome, noise], outcomes)

    def build_discriminator(self):

        model = Sequential()

        model.add(Dense(512, input_dim=self.pat_features+self.num_treatments))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dense(512,activation='relu'))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dense(512,activation='sigmoid'))
        model.add(Dense(512,activation='relu'))
        model.add(Dense(self.num_treatments, activation='sigmoid', name='treatment_probs'))

        pat = Input(shape=(self.pat_features,), dtype='float32')
        
        y_bar = Input(shape=(self.num_treatments,), dtype='float32')   
        
        model_input = concatenate([pat,y_bar])
        
        treatment_probs = model(model_input)
                          
        return Model([pat, y_bar], treatment_probs)
    

    def train(self, epochs, split_date='2015-11-01', batch_size=BATCH_SIZE):
        
        # Load the dataset
        X_train, y_train, t_train, _ = self.data.train[split_date]
        
        # Configure input
        X_train = X_train.values.astype(np.float32)
        y_train = y_train.reshape(-1,1)
        t_train = t_train.reshape(-1,self.num_treatments)
                
        for epoch in range(epochs):

            # ---------------------
            #  Train Discriminator
            # ---------------------

            # Select a random batch of patients
            idx = np.random.randint(0, X_train.shape[0], batch_size)
            pats, treatments, real_outcomes = X_train[idx], t_train[idx], y_train[idx]
            
            treatment_indices = np.argmax(treatments,axis=1).reshape(-1,1).astype(np.int64)
            noise = np.random.uniform(-1, 2, (batch_size, self.latent_dim))
                        
            # Generate y | (X,t,y_f)
            gen_y = self.generator.predict([pats, treatments, real_outcomes, noise])

            # ---------------------
            #  Train Generator
            # ---------------------
            
            # Train the generator
            g_loss = self.combined.train_on_batch([pats, treatments, real_outcomes, noise, treatment_indices], [treatments, real_outcomes])
            
            # Plot the progress
            if epoch % 200 == 0:
                print ("{}: D loss {}, G loss: {}".format(epoch, g_loss[1], g_loss[2]))            

        self.trained=datetime.datetime.now()
        
    
    def generate_complete_dataset(self):

        X = self.data.df.drop(columns=['dialysis','mort12mo']).values
        y = self.data.df['mort12mo'].values
        t = one_hot(self.data.df['dialysis'].values)

        treatment_indices = np.argmax(t,axis=1)
        noise = np.random.uniform(-1, 2, (X.shape[0], self.latent_dim))

        # Generate y | (X,t,y_f)
        gen_y = self.generator.predict([X, t, y, noise])

        y_bar = gen_y.copy()
        
        for k in range(gen_y.shape[0]):
            y_bar[k,treatment_indices[k]]=y[k]
        
        self.data.complete_fake_y = gen_y
        self.data.complete_real_y = y_bar
        
# if __name__ == '__main__':
#     cgan = CGAN()
#     cgan.train(epochs=20000, batch_size=32, sample_interval=200)

In [35]:
K.clear_session()
gdg = G_DG(datas[0])
gdg.train(epochs=10000)

TypeError: Value passed to parameter 'indices' has DataType float32 not in list of allowed values: int32, int64

In [79]:
gdg.generate_complete_dataset()

In [ ]:
class I_DI():
    def __init__(self,data):
        
        self.data = data
        self.pat_features = self.data.n_features
        self.pat_shape = (self.pat_features,)
        self.num_treatments = 2
        self.latent_dim = self.num_treatments
        self.b = 2.
        
        
        self.optimizer = Adam(0.0002, 0.5)
        
#         self.check_intermediate_layers()
        self.build_model()

    def check_intermediate_layers(self):
        # Build and compile the discriminator
        self.discriminator = self.build_discriminator()
        self.discriminator.compile(loss=['binary_crossentropy'],
                                   optimizer=self.optimizer)

        # Build the generator
        self.generator = self.build_generator()

        pat = Input(shape=(self.pat_features,))
        noise = Input(shape=(self.latent_dim,))
        
        gen_y = self.generator([pat, treatment, outcome, noise])                
        
        y_bar = Lambda(self.replace_values)([gen_y, treatment_indices, outcome])
        
        z=Model([pat,noise,treatment_indices],gen_y)
        zz=Model([pat,noise,treatment_indices],y_bar)
        
        pat = np.arange(self.pat_features * BATCH_SIZE).reshape((BATCH_SIZE,self.pat_features))
        noise = np.random.uniform(-1,2,(BATCH_SIZE,self.latent_dim))

        gen_y = z.predict([pat,treatment,outcome,noise,treatment_indices])
        ybar = zz.predict([pat,treatment,outcome,noise,treatment_indices])
        
    def build_model(self):
        
        self.discriminator = self.build_discriminator()        
        self.generator = self.build_generator()

        pat = Input(shape=(self.pat_features,))
        noise = Input(shape=(self.latent_dim,))
        treatment_indices = Input(shape=(1,))
        outcome = Input(shape=(1,))
        real_indicator = Input(shape=(1,))

        gen_y = self.generator([pat, noise])
        
        predicted_outcome_of_real_treatment = Lambda(get_values,name='predicted_outcome_of_real_treatment')([gen_y,treatment_indices])

        y_bar = Lambda(self.replace_values,name='y_bar')([gen_y, treatment_indices, outcome])

        y = K.switch(K.equal(real_indicator,0),gen_y,y_bar)
    
        pred_real = self.discriminator([pat, y])        
        
        self.combined = Model([pat, noise, treatment_indices, outcome, real_indicator], [pred_real, predicted_outcome_of_real_treatment])
        
        self.combined.compile(loss=['binary_crossentropy','mean_squared_error'],
                              loss_weights=[1.,self.b],
                              optimizer=self.optimizer)
   

    def replace_values(self,x):
        outs, indices, values = x

        #this is due to a strange bug between lambda and integers....
        indices = K.cast(indices, 'int32')

        #create one_hot indices
        one_hot_indices = K.one_hot(indices, self.num_treatments) #size is the size of gen_out
        one_hot_indices = K.batch_flatten(one_hot_indices)

        #have the desired values at their correct positions
        values_to_use = one_hot_indices * values

        #if values are 0, use gen_out, else use values
        return K.switch(K.equal(values_to_use, 0), outs, values_to_use)
        
    def build_generator(self):

        model = Sequential()

        model.add(Dense(512, input_dim=self.pat_features+self.latent_dim))
        model.add(LeakyReLU(alpha=0.2))
        model.add(BatchNormalization(momentum=0.6))
        model.add(Dense(1024))
        model.add(LeakyReLU(alpha=0.2))
        model.add(BatchNormalization(momentum=0.6))
        model.add(Dense(256))
        model.add(LeakyReLU(alpha=0.2))
        model.add(BatchNormalization(momentum=0.6))  
        model.add(Dense(self.num_treatments, activation='hard_sigmoid'))

        pat = Input(shape=(self.pat_features,),dtype='float32')
        noise = Input(shape=(self.latent_dim,),dtype='float32')

        model_input = concatenate([pat, noise])

        outcomes = model(model_input)
        
        return Model([pat, noise], outcomes)

    def build_discriminator(self):

        model = Sequential()

        model.add(Dense(512, input_dim=self.pat_features+self.num_treatments))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dense(512,activation='relu'))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dense(512,activation='sigmoid'))
        model.add(Dense(512,activation='relu'))
        model.add(Dense(1, activation='sigmoid', name='real_prob'))

        pat = Input(shape=(self.pat_features,), dtype='float32')
        outcome = Input(shape=(self.num_treatments,), dtype='float32')   
        
        model_input = concatenate([pat,outcome])
        
        treatment_probs = model(model_input)
                          
        return Model([pat, outcome], treatment_probs)
    

    def train(self, epochs, split_date='2017-11-01', batch_size=BATCH_SIZE):
        
        # Load the dataset
        X_train, y_train, t_train, _ = self.data.train[split_date]
        
        # Configure input
        X_train = X_train.values.astype(np.float32)
        y_train = y_train.reshape(-1,1)
        t_train = t_train.reshape(-1,self.num_treatments)
                
        for epoch in range(epochs):

            # ---------------------
            #  Train Discriminator
            # ---------------------

            # Select a random batch of patients
            idx = np.random.randint(0, X_train.shape[0], batch_size)
            pats, treatments, real_outcomes = X_train[idx], t_train[idx], y_train[idx]
            
            real_indicators = np.random.randint(0,2,batch_size).reshape(-1,1)
            
            treatment_indices = np.argmax(treatments,axis=1).reshape(-1,1)
            noise = np.random.uniform(-1, 2, (batch_size, self.latent_dim))
                        
            # Generate y | (X,t,y_f)
            gen_y = self.generator.predict([pats, noise])

            # ---------------------
            #  Train Generator
            # ---------------------

            loss = self.combined.train_on_batch([pats, noise, treatment_indices, outcome, real_indicators], [real_indicators, real_outcomes])
            
            # Plot the progress
            if epoch % 200 == 0:
                print ("{} [sum, dis, gen] loss: {}".format(epoch, loss))            